In [2]:
import pandas as pd

In [3]:
# Carregar a base de óbitos
obitos_df = pd.read_csv("Agregados_por_distritos_obitos_BR.csv", encoding="latin1", sep=";")

In [4]:
# Carregar a base com área dos distritos
basico_df = pd.read_csv("Agregados_por_distritos_basico_BR.csv", encoding="latin1", sep=";")

In [5]:
# Remover espaços extras dos nomes das colunas
obitos_df.columns = obitos_df.columns.str.strip()
basico_df.columns = basico_df.columns.str.strip()

In [6]:
# Renomear colunas da base
obitos_df.rename(columns={'V01238': 'M >= 70 ANOS', 'V01239': 'F >= 70 ANOS'}, inplace=True)
basico_df.rename(columns={'v0001': 'POP_TOTAL'}, inplace=True)

In [7]:
# Exibir os nomes das colunas disponíveis em cada DataFrame (opcional para debug)
print("Colunas em obitos_df:", obitos_df.columns.tolist())
print("Colunas em basico_df:", basico_df.columns.tolist())

Colunas em obitos_df: ['CD_DIST', 'NM_DIST', 'V01224', 'V01225', 'V01226', 'V01227', 'V01228', 'V01229', 'V01230', 'V01231', 'V01232', 'V01233', 'V01234', 'V01235', 'V01236', 'V01237', 'M >= 70 ANOS', 'F >= 70 ANOS', 'V01240', 'V01241', 'V01242', 'V01243', 'V01244', 'V01245', 'V01246', 'V01247', 'V01248', 'V01249', 'V01250', 'V01251', 'V01252', 'V01253', 'V01254', 'V01255', 'V01256', 'V01257', 'V01258', 'V01259', 'V01260', 'V01261', 'V01262', 'V01263', 'V01264', 'V01265', 'V01266', 'V01267', 'V01268', 'V01269', 'V01270', 'V01271', 'V01272', 'V01273', 'V01274', 'V01275', 'V01276', 'V01277', 'V01278', 'V01279', 'V01280', 'V01281', 'V01282', 'V01283', 'V01284', 'V01285', 'V01286', 'V01287', 'V01288', 'V01289', 'V01290', 'V01291', 'V01292', 'V01293', 'V01294', 'V01295', 'V01296', 'V01297', 'V01298', 'V01299', 'V01300', 'V01301', 'V01302', 'V01303', 'V01304', 'V01305', 'V01306', 'V01307', 'V01308', 'V01309', 'V01310', 'V01311', 'V01312', 'V01313', 'V01314', 'V01315', 'V01316']
Colunas em ba

In [8]:
# Selecionar colunas desejadas da base de óbitos
obitos_selecionados = obitos_df[["CD_DIST", "NM_DIST", "M >= 70 ANOS", "F >= 70 ANOS"]]

In [10]:
# Selecionar colunas da base com área (confirmando que existem)
area_selecionada = basico_df[["CD_DIST", "AREA_KM2", "POP_TOTAL"]]

In [11]:
# Realizar merge
df_merged = pd.merge(obitos_selecionados, area_selecionada, on=["CD_DIST"], how="left")

In [12]:
# Exportar para CSV com codificação UTF-8
df_merged.to_csv("obitos_normalizados.csv", index=False, encoding='utf-8')
print("Arquivo 'obitos_normalizados.csv' criado com sucesso.")

Arquivo 'obitos_normalizados.csv' criado com sucesso.
